In [ ]:
import subprocess
subprocess.check_call(["pip", "install", "selenium"])
subprocess.check_call(["pip", "install", "webdriver-manager"])
subprocess.check_call(["pip", "install", "requests"])

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
import requests
import os

In [3]:
def abrir_navegador():
    """
    Abre o navegador e acessa o site do IBGE.

    :return: Navegador aberto.
    """
    opcoes = Options()
    opcoes.add_argument('--headless') # Rodar sem abrir o navegador
    # opcoes.add_argument('--auto-open-devtools-for-tabs') # Abrir o console do navegador
    # opcoes.add_argument('--start-maximized')
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    return navegador

def fechar_navegador(navegador):
    """
    Fecha o navegador.

    :param navegador: Navegador aberto.
    """
    navegador.quit()

def retornar_tabela(navegador):
    """
    Retorna a tabela com os dados de população por município do estado de São Paulo.

    :return: Tabela com os dados do site do IBGE.
    """
    elemento = (By.XPATH, '//*[@id="municipios"]/tbody[1]/tr/td[1]')
    WebDriverWait(navegador, 30).until(EC.presence_of_element_located(elemento))
    elemento = (By.XPATH, '//*[@id="municipios"]')
    tabela = WebDriverWait(navegador, 30).until(EC.presence_of_element_located(elemento))
    return tabela

def acessar_site(navegador, url):
    """
    Acessa o site pela URL.

    :param navegador: Navegador aberto.
    :param url: URL do site do IBGE.
    """
    navegador.get(url)
    tabela = retornar_tabela(navegador)
    return tabela


In [4]:
def dados_populacao(navegador):
    """
    Retorna um data frame com os dados de população por município do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site(navegador, 'https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=96385,96386,29765,29763,60036,47001')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.drop(df.index[-1], inplace=True)
    df.rename(columns={'Municípios': 'cidade',
                       'População no último censo': 'populacao_censo',
                       'Pessoal ocupado': 'populacao_ocupado',
                       'População ocupada': 'porcentagem_populacao_ocupado',
                       'Salário médio mensal dos trabalhadores formais': 'salario_medio_mensal_trabalhadores_formais',
                       'PIB per capita': 'pib_per_capita',
                       'Densidade demográfica': 'densidade_demografica'}, inplace=True)
    df['cidade'] = df['cidade'].astype(str)
    df['populacao_censo'] = df['populacao_censo'].str.replace(' pessoas', '').astype(int)
    df['populacao_ocupado'] = df['populacao_ocupado'].str.replace(' pessoas', '').astype(int)
    df['porcentagem_populacao_ocupado'] = df['porcentagem_populacao_ocupado'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['salario_medio_mensal_trabalhadores_formais'] = df['salario_medio_mensal_trabalhadores_formais'].str.replace(' salários mínimos', '').str.replace(',', '.').astype(float)
    df['pib_per_capita'] = df['pib_per_capita'].str.replace(' R$', '').str.replace(',', '.').astype(float)
    df['densidade_demografica'] = df['densidade_demografica'].str.replace(' habitante por quilômetro quadrado', '').str.replace(',', '.').astype(float)
    return df

def dados_cidade(navegador):
    """
    Retorna um data frame com os dados referentes a municípios do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site(navegador, 'https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=30255,95335,60029,60031,29167')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.drop(df.index[-1], inplace=True)
    df.rename(columns={'Municípios': 'cidade',
                       'Área da unidade territorial': 'area_territorio',
                       'Área urbanizada': 'area_urbanizada',
                       'Urbanização de vias públicas': 'urbanizacao_vias_publicas',
                       'Arborização de vias públicas': 'arborizacao_vias_publicas',
                       'Índice de Desenvolvimento Humano Municipal (IDHM)': 'idhm'}, inplace=True)
    df['cidade'] = df['cidade'].astype(str)
    df['area_territorio'] = df['area_territorio'].str.replace(' km²', '').str.replace(',', '.').astype(float)
    df['area_urbanizada'] = df['area_urbanizada'].str.replace(' km²', '').str.replace(',', '.').astype(float)
    df['urbanizacao_vias_publicas'] = df['urbanizacao_vias_publicas'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['arborizacao_vias_publicas'] = df['arborizacao_vias_publicas'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['idhm'] = (df['idhm'].astype(float) * 0.001).round(3)
    return df

def dados_regiao():
    """
    Retorna um data frame com os dados referentes a regiões do estado de São Paulo.

    :return: Data frame com os dados de microrregiões, mesorregiões, regiões imediatas e regiões intermediárias do estado de São Paulo.
    """
    try:
        response = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios/')

        if response.status_code == 200:
            tabela = response.json()

            df = pd.json_normalize(tabela)
            df = df[['nome', 'microrregiao.nome', 'microrregiao.mesorregiao.nome', 'regiao-imediata.nome', 'regiao-imediata.regiao-intermediaria.nome']]
            df.rename(columns={
                'nome': 'cidade',
                'microrregiao.nome': 'microrregiao',
                'microrregiao.mesorregiao.nome': 'mesorregiao',
                'regiao-imediata.nome': 'regiao_imediata',
                'regiao-imediata.regiao-intermediaria.nome': 'regiao_intermediaria'}, inplace=True)
            df['cidade'] = df['cidade'].astype(str)
            df['microrregiao'] = df['microrregiao'].astype(str)
            df['mesorregiao'] = df['mesorregiao'].astype(str)
            df['regiao_imediata'] = df['regiao_imediata'].astype(str)
            df['regiao_intermediaria'] = df['regiao_intermediaria'].astype(str)
            return df
        else:
            print(f"A solicitação falhou com status {response.status_code}")
    except Exception as e:
        print(f"Ocorreu um erro durante a solicitação: {e}")
    return None

def dados_cidade_preenchida_errada():
    """
    Retorna um data frame com os dados referentes a cidades do estado de São Paulo que foram preenchidos errados.

    :return: Data frame com os dados de cidades do estado de São Paulo que foram preenchidos errados.
    """
    erradas = {
        "Aparecida d'Oeste": 'APARECIDA D OESTE',
        'Biritiba Mirim': 'BIRITIBA-MIRIM',
        'Brodowski': 'BRODÓSQUI',
        'Campina do Monte Alegre': 'CAMPINA MONTE ALEGRE',
        'Campos Novos Paulista': 'CAMPOS NOVOS PAUL.',
        'Embu-Guaçu': 'EMBU,EMBU GUAÇÚ',
        'Espírito Santo do Turvo': 'ESPIRITO S.DO TURVO',
        'Espírito Santo do Pinhal': 'ESPIRITO STO. PINHAL',
        "Estrela d'Oeste": 'ESTRELA D OESTE',
        'Euclides da Cunha Paulista': 'EUCLIDES DA CUNHA PTA',
        'Florínea': 'FLORINIA',
        "Guarani d'Oeste": 'GUARANI D OESTE,GUARANI DOESTE',
        'Ipaussu': 'IPAUCU,IPAUÇU',
        'Mirante do Paranapanema': 'MIRANTE PARANAPANEMA',
        'Mogi Guaçu': 'MOGIGUACU,MOGI-GUACU,MOJI-GUACU',
        'Mogi Mirim': 'MOGIMIRIM,MOGI-MIRIM',
        "Palmeira d'Oeste": 'PALMEIRA D OESTE,PALMEIRA DOESTE',
        'Pirapora do Bom Jesus': 'PIRAPORA BOM JESUS',
        'Ribeirão dos Índios': 'RIBEIRAO INDIOS',
        'Santa Adélia': 'S.ADELIA',
        'Santa Albertina': 'S.ALBERTINA',
        'Santo Anastácio': 'S.ANASTACIO',
        'Santo André': 'S.ANDRE',
        'Santo Antônio da Alegria': 'S.ANTONIO DA ALEGRIA',
        'Santo Antônio do Aracanguá': 'S.ANTONIO DE ARACANGUA,SANTO ANTÔNIO DE ARACANGUÁ',
        'Santo Antônio de Posse': 'S.ANTONIO DE POSSE,SANTO ANTÔNIO DA POSSE',
        'Santo Antônio do Jardim': 'S.ANTONIO DO JARDIM',
        'Santo Antônio do Pinhal': 'S.ANTONIO DO PINHAL',
        "Santa Bárbara d'Oeste": 'S.BARBARA D OESTE,SANTA BÁRBARA D OESTE,SANTA BÁRBARA DOESTE',
        'São Bento do Sapucaí': 'S.BENTO DO SAPUCAI',
        'São Bernardo do Campo': 'S.BERNARDO DO CAMPO',
        'Santa Branca': 'S.BRANCA',
        'São Caetano do Sul': 'S.CAETANO DO SUL',
        'São Carlos': 'S.CARLOS',
        'Santa Cruz da Conceição': 'S.CRUZ DA CONCEICAO',
        'Santa Cruz da Esperança': 'S.CRUZ DA ESPERANCA',
        'Santa Cruz das Palmeiras': 'S.CRUZ DAS PALMEIRAS',
        'Santa Cruz do Rio Pardo': 'S.CRUZ DO RIO PARDO',
        'Santa Ernestina': 'S.ERNESTINA',
        'Santo Expedito': 'S.EXPEDITO',
        'Santa Fé do Sul': 'S.FE DO SUL',
        'São Francisco': 'S.FRANCISCO',
        'Santa Gertrudes': 'S.GERTRUDES',
        'Santa Isabel': 'S.ISABEL',
        'São João da Boa Vista': 'S.JOAO DA BOA VISTA',
        'São João de Iracema': 'S.JOAO DE IRACEMA',
        "São João do Pau d'Alho": 'S.JOAO DO PAU D ALHO,SÃO JOÃO DO PAU D ALHO,SÃO JOÃO DO PAU DALHO',
        'São João das Duas Pontes': 'S.JOAO DUAS PONTES',
        'São Joaquim da Barra': 'S.JOAQUIM DA BARRA',
        'São José da Bela Vista': 'S.JOSE DA BELA VISTA',
        'São José do Barreiro': 'S.JOSE DO BARREIRO',
        'São José do Rio Pardo': 'S.JOSE DO RIO PARDO',
        'São José do Rio Preto': 'S.JOSE DO RIO PRETO',
        'São José dos Campos': 'S.JOSE DOS CAMPOS',
        'São Lourenço da Serra': 'S.LOURENCO DA SERRA',
        'Santa Lúcia': 'S.LUCIA',
        'São Luiz do Paraitinga': 'S.LUIZ DO PARAITINGA,S.LUIS DO PARAITINGA,SÃO LUÍS DO PARAITINGA',
        'São Manuel': 'S.MANUEL',
        'Santa Maria da Serra': 'S.MARIA DA SERRA',
        'Santa Mercedes': 'S.MERCEDES',
        'São Miguel Arcanjo': 'S.MIGUEL ARCANJO',
        'São Paulo': 'S.PAULO',
        'São Pedro do Turvo': 'S.PEDRO DO TURVO',
        'São Pedro': 'S.PEDRO',
        "Santa Rita d'Oeste": 'S.RITA D OESTE',
        'Santa Rita do Passa Quatro': 'S.RITA PASSA QUATRO',
        'São Roque': 'S.ROQUE',
        'Santa Rosa de Viterbo': 'S.ROSA DE VITERBO,SANTA ROSA DO VITERBO',
        'Santa Salete': 'S.SALETE',
        'São Sebastião da Grama': 'S.SEBASTIAO DA GRAMA',
        'São Sebastião': 'S.SEBASTIAO',
        'São Simão': 'S.SIMAO',
        'São Vicente': 'S.VICENTE',
        'Salmourão': 'SALMORAO',
        "Santa Clara d'Oeste": 'SANTA CLARA DOESTE,S.CLARA D OESTE',
        'Sebastianópolis do Sul': 'SEBASTIANOPOLIS SUL',
        'Sud Mennucci': 'SUD MENUCCI',
        'Três Lagoas': 'TRES LAGOAS'
    }
    df = pd.DataFrame(list(erradas.items()), columns=['cidade', 'cidade_preenchida_errada'])
    df['cidade'] = df['cidade'].astype(str)
    df['cidade_preenchida_errada'] = df['cidade_preenchida_errada'].astype(str)
    return df

def coletar_dados():
    """
    Coleta os dados por município do estado de São Paulo.

    :return: Data frame com os dados por município do estado de São Paulo.
    """
    navegador = abrir_navegador()
    df_populacao = dados_populacao(navegador)
    df_cidade = dados_cidade(navegador)
    fechar_navegador(navegador)

    df_regiao = dados_regiao()
    df_cidade_preenchida_errada = dados_cidade_preenchida_errada()

    df_unido = pd.merge(df_populacao, df_cidade, on='cidade')
    df_unido2 = pd.merge(df_unido, df_regiao, on='cidade')
    df = pd.merge(df_unido2, df_cidade_preenchida_errada, on='cidade', how="left")
    df = df[['cidade', 'cidade_preenchida_errada', 'regiao_intermediaria', 'regiao_imediata', 'mesorregiao', 'microrregiao',
            'populacao_censo', 'populacao_ocupado', 'porcentagem_populacao_ocupado', 'salario_medio_mensal_trabalhadores_formais', 'pib_per_capita',
            'area_territorio', 'densidade_demografica', 'area_urbanizada', 'urbanizacao_vias_publicas', 'arborizacao_vias_publicas', 'idhm']]
    return df

def exportar_dados(dataframe):
    """
    Exporta os dados para um arquivo CSV.
    """
    nome_arquivo = 'dados_cidades.csv'
    diretorio = os.path.join(os.getcwd(), os.path.abspath('..\\'), 'Analises', 'Dados', nome_arquivo)
    dataframe.to_csv(diretorio, sep=';', encoding='latin1')


In [5]:
df = coletar_dados()
exportar_dados(df)
df.head(1000)

C:\Users\ander\AppData\Local\Temp\ipykernel_23988\4133633098.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
C:\Users\ander\AppData\Local\Temp\ipykernel_23988\4133633098.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]


,cidade,cidade_preenchida_errada,regiao_intermediaria,regiao_imediata,mesorregiao,microrregiao,populacao_censo,populacao_ocupado,porcentagem_populacao_ocupado,salario_medio_mensal_trabalhadores_formais,pib_per_capita,area_territorio,densidade_demografica,area_urbanizada,urbanizacao_vias_publicas,arborizacao_vias_publicas,idhm
0,Adamantina,NaN,Presidente Prudente,Adamantina - Lucélia,Presidente Prudente,Adamantina,34687,12303,33.7,2.2,35788.26,411.987,84.19,12.43,17.9,99.3,0.790
1,Adolfo,NaN,São José do Rio Preto,São José do Rio Preto,São José do Rio Preto,São José do Rio Preto,4351,738,22.2,1.8,38464.09,211.055,20.62,2.33,4.9,98.6,0.730
2,Aguaí,NaN,Campinas,São João da Boa Vista,Campinas,Pirassununga,32072,7429,18.7,2.3,31446.28,474.554,67.58,6.94,14.7,91.1,0.715
3,Águas da Prata,NaN,Campinas,São João da Boa Vista,Campinas,São João da Boa Vista,7369,1167,13.8,2.2,22718.42,142.673,51.65,3.27,46.1,81.4,0.781
4,Águas de Lindóia,NaN,Campinas,Amparo,Campinas,Amparo,17930,6138,30.5,1.9,26418.38,60.126,298.21,8.53,73.3,82.1,0.745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,Vista Alegre do Alto,NaN,São José do Rio Preto,Catanduva,Ribeirão Preto,Jaboticabal,8109,3367,35.7,2.9,44966.33,95.429,84.97,2.03,44.2,98.1,0.744
641,Vitória Brasil,NaN,São José do Rio Preto,Jales,São José do Rio Preto,Jales,1794,247,14.5,2.2,23045.31,49.832,36.00,0.51,5.8,99.6,0.725
642,Votorantim,NaN,Sorocaba,Sorocaba,Macro Metropolitana Paulista,Sorocaba,127923,23262,18.3,2.6,27713.57,184.186,694.53,25.54,58.2,83.4,0.767
643,Votuporanga,NaN,São José do Rio Preto,Votuporanga,São José do Rio Preto,Votuporanga,96634,29941,31.3,2.2,35390.26,420.703,229.70,25.65,50.8,97.2,0.790
